## Clustering Assignment 

## Submitted By : Kanika Kathpalia


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import cut_tree

In [ ]:
# Reading the Datset


country_df= pd.read_csv("Country-data.csv")
country_df.head()

## Understanding the data

In [ ]:
country_df.info()

#### Observation: All Columns are numnerical except Country

## Data Preparation

#### Cleaning /Handling Missing Data

In [ ]:
## Checking Missing Values:

country_df.isnull().sum()

#### Observation: No missing values, hence we can proceed with our next step for data analysis

In [ ]:
# Segregation of Numerical and Categorical Variables/Columns

cat_col = country_df.select_dtypes(include = ['object']).columns
num_col = country_df.select_dtypes(exclude = ['object']).columns

In [ ]:
# Converting exports,imports and health spending percentages to absolute values.

country_df['exports'] = country_df['exports'] * country_df['gdpp']/100
country_df['imports'] = country_df['imports'] * country_df['gdpp']/100
country_df['health'] = country_df['health'] * country_df['gdpp']/100

In [ ]:
country_df.head()

## Outlier Treatment

In [ ]:
country_df.describe()

In [ ]:
## Checking Outliers

plt.figure(figsize=(18, 10))
plt.subplot(3, 3, 1)
sns.boxplot(country_df.child_mort)
plt.subplot(3, 3, 2)
sns.boxplot(country_df.exports)
plt.subplot(3, 3, 3)
sns.boxplot(country_df.health)
plt.subplot(3, 3, 4)
sns.boxplot(country_df.imports)
plt.subplot(3, 3, 5)
sns.boxplot(country_df.income)
plt.subplot(3, 3, 6)
sns.boxplot(country_df.inflation)
plt.subplot(3, 3, 7)
sns.boxplot(country_df.life_expec)
plt.subplot(3, 3, 8)
sns.boxplot(country_df.total_fer)
plt.subplot(3, 3, 9)
sns.boxplot(country_df.gdpp)
plt.show()

#### Observation: There seemes to be outliers in every single variable. This is a very delicate situation in terms of Business problem statement & Clustering analysis. - If we apply outlier treatment by CAPPING this will change the ranking of few countries with respect to requirement of Financial Aid. Also we will still have some outlier present after Capping, so its not a wise decision in this business scenario. - If we apply outlier treatment by Deletion based on IQR values, this will remove few countries from the list that would have really deserved the Financial Aid. - If we do not apply Outlier treatment, it can impact the clustering model, as the presence of Outlier can change the CENTROID (K-Means) of the cluster. - After considering all these scenarios, we decided to go with the model which suites the Business Problem better. That is, not to treat the Outlier, and check the model with different K values to see which one gives a better business outcome.

## Data Visualisation

In [ ]:
country_df.columns

In [ ]:
## Pairplot:

sns.pairplot(country_df)
plt.show()

In [ ]:
plt.figure(figsize=(21, 18))
features = ['child_mort', 'exports', 'health', 'imports', 'income','inflation', 'life_expec', 'total_fer', 'gdpp']
for i in enumerate(features):
    ax = plt.subplot(3, 3, i[0]+1)
    sns.distplot(country_df[i[1]])
    plt.xticks(rotation=20)

#### Observation: Most of the data points are 'NOT Normally' distributed. - Their variance are also differernt. - Their range are also differnt . All the above points indicates the need of standardising the data before we build the model. Since we need to compute the Euclidean distance between the data points, it is important to ensure that the attributes with a larger range of values do not out-weight the attributes with smaller range. Thus, scaling down of all attributes to the same normal scale is important here.

## Scaling the Dataset

In [ ]:

# New dataframe with only Numeric variables, for scaling

country_df_new=country_df[['child_mort', 'exports', 'health', 'imports', 'income',
       'inflation', 'life_expec', 'total_fer', 'gdpp']].copy()

In [ ]:
# Scaling the dataset with Standard Scaler 

scaler = StandardScaler()
country_df_scaled = scaler.fit_transform(country_df_new)

In [ ]:
country_df_scaled

In [ ]:
#Converting it into a dataframe

country_df_scaled = pd.DataFrame(country_df_scaled)
country_df_scaled.columns = ['child_mort', 'exports', 'health', 'imports', 'income',
       'inflation', 'life_expec', 'total_fer', 'gdpp']
country_df_scaled.head()

## Hopkins Statistics

In [ ]:
from sklearn.neighbors import NearestNeighbors
from random import sample
from numpy.random import uniform
import numpy as np
from math import isnan
 
def hopkins(X):
    d = X.shape[1]
    #d = len(vars) # columns
    n = len(X) # rows
    m = int(0.1 * n) 
    nbrs = NearestNeighbors(n_neighbors=1).fit(X.values)
 
    rand_X = sample(range(0, n, 1), m)
 
    ujd = []
    wjd = []
    for j in range(0, m):
        u_dist, _ = nbrs.kneighbors(uniform(np.amin(X,axis=0),np.amax(X,axis=0),d).reshape(1, -1), 2, return_distance=True)
        ujd.append(u_dist[0][1])
        w_dist, _ = nbrs.kneighbors(X.iloc[rand_X[j]].values.reshape(1, -1), 2, return_distance=True)
        wjd.append(w_dist[0][1])
 
    H = sum(ujd) / (sum(ujd) + sum(wjd))
    if isnan(H):
        print(ujd, wjd)
        H = 0
 
    return H

In [ ]:
#Use the Hopkins Statistic function by passing the above dataframe as a paramter
hopkins(country_df_scaled)

### Observation: Hopkins Statistic over .80 is a good score that indicated that the data is good for cluster analysis. 

## Modelling

## K Means Clustering
## Finding the Optimal Number of Clusters

### SSD (Elbow Curve)

In [ ]:
country_df_scaled.shape

In [ ]:
# elbow-curve/SSD
ssd = []
range_n_clusters = [2, 3, 4, 5, 6, 7, 8]
for num_clusters in range_n_clusters:
    kmeans = KMeans(n_clusters=num_clusters, max_iter=50)
    kmeans.fit(country_df_scaled)
    
    ssd.append(kmeans.inertia_)
    
# plot the SSDs for each n_clusters
# ssd
plt.plot(ssd)

###  Silhouette Analysis

In [ ]:
range_n_clusters = [2, 3, 4, 5, 6, 7, 8]

for num_clusters in range_n_clusters:
    
    # intialise kmeans
    kmeans = KMeans(n_clusters=num_clusters, max_iter=50)
    kmeans.fit(country_df_scaled)
    
    cluster_labels = kmeans.labels_
    
    # silhouette score
    silhouette_avg = silhouette_score(country_df_scaled, cluster_labels)
    print("For n_clusters={0}, the silhouette score is {1}".format(num_clusters, silhouette_avg))

 ### Observation:From the above validations(Elbow Curve & silhouette analysis), we could see that 3,4 or 5 clusters are optimal number of clusters to be used. We will try k=3,4,5 and analyse the results.

## K-Means Clustering
### Using k= 3 

In [ ]:
# K Means with with k=3
kmeans_3 = KMeans(n_clusters=3, max_iter=500, init='k-means++', n_init=10, random_state= 335)
kmeans_3.fit(country_df_scaled)

In [ ]:
kmeans_3.labels_

In [ ]:
# Create a new datafame with scaled data and adding country name to it.
ngo_country_k = country_df.copy()
ngo_country_k.head()

In [ ]:
# assign the label
ngo_country_k['cluster_id_3'] = kmeans_3.labels_
ngo_country_k.head()

In [ ]:
# Checking the number of countries in each cluster

ngo_country_k.cluster_id_3.value_counts()

### Visualizing the Clustered Data

In [ ]:
plt.figure(figsize=(18, 5))
plt.subplot(1, 3, 1)
sns.boxplot(x='cluster_id_3', y='child_mort',
            data=ngo_country_k, palette="bright")
plt.subplot(1, 3, 2)
sns.boxplot(x='cluster_id_3', y='gdpp',
            data=ngo_country_k, palette="bright")
plt.subplot(1, 3, 3)
sns.boxplot(x='cluster_id_3', y='income',
            data=ngo_country_k, palette="bright")

plt.show()

In [ ]:

plt.figure(figsize=(18, 5))
plt.subplot(1, 3, 1)
sns.scatterplot(x='gdpp', y='child_mort', hue='cluster_id_3',
                data=ngo_country_k, legend='full', palette="bright")
plt.subplot(1, 3, 2)
sns.scatterplot(x='income', y='child_mort', hue='cluster_id_3',
                data=ngo_country_k, legend='full', palette="bright")
plt.subplot(1, 3, 3)
sns.scatterplot(x='gdpp', y='income', hue='cluster_id_3',
                data=ngo_country_k, legend='full', palette="bright")
plt.show()

In [ ]:
# Checking the CLuster means:
ngo_country_k.groupby(['cluster_id_3']).mean().sort_values(['child_mort','income','gdpp'],ascending = [False,True,True])

### Observation: Cluster 2 & Cluster 0 seems to have highest Child Mortality and lowest Income & GDPP. Lets check the countries in these clusters to see there outcome.

In [ ]:
# Checking the countries in Cluster 2 to see which are the countries in that segment.

ngo_country_k[['country','child_mort','income','gdpp']][ngo_country_k['cluster_id_3'] == 2].sort_values(['child_mort','income','gdpp'],ascending = [False,True,True])

## Second Iteration - Using k= 4

In [ ]:
# K Means with with k=4
kmeans_4 = KMeans(n_clusters=4, max_iter=500, init='k-means++', n_init=10, random_state= 335)
kmeans_4.fit(country_df_scaled)

In [ ]:
kmeans_4.labels_

In [ ]:
# assign the label
ngo_country_k['cluster_id_4'] = kmeans_4.labels_
ngo_country_k.head()

In [ ]:
# Checking the number of countries in each cluster

ngo_country_k.cluster_id_4.value_counts()

In [ ]:
## Visualisation

# Box plot on various variable against the CLUSTER_ID to visualize the spread of the data

plt.figure(figsize=(18, 5))
plt.subplot(1, 3, 1)
sns.boxplot(x='cluster_id_4', y='child_mort',
            data=ngo_country_k, palette="bright")
plt.subplot(1, 3, 2)
sns.boxplot(x='cluster_id_4', y='gdpp',
            data=ngo_country_k, palette="bright")
plt.subplot(1, 3, 3)
sns.boxplot(x='cluster_id_4', y='income',
            data=ngo_country_k, palette="bright")

plt.show()

In [ ]:
# Scatter plot on various variables to visualize the clusters based on them

plt.figure(figsize=(18, 5))
plt.subplot(1, 3, 1)
sns.scatterplot(x='gdpp', y='child_mort', hue='cluster_id_4',
                data=ngo_country_k, legend='full', palette="bright")
plt.subplot(1, 3, 2)
sns.scatterplot(x='income', y='child_mort', hue='cluster_id_4',
                data=ngo_country_k, legend='full', palette="bright")
plt.subplot(1, 3, 3)
sns.scatterplot(x='gdpp', y='income', hue='cluster_id_4',
                data=ngo_country_k, legend='full', palette="bright")
plt.show()

In [ ]:
ngo_country_k.groupby(['cluster_id_4']).mean().sort_values(['child_mort','income','gdpp'],ascending = [False,True,True])

### Observation: Cluster 3 & Cluster 1 seems to have highest Child Mortality and lowest Income & GDPP. 

## Third Iteration - Using k=5

In [ ]:
# K Means with with k=5
kmeans_5 = KMeans(n_clusters=5, max_iter=500, init='k-means++', n_init=10, random_state= 335)
kmeans_5.fit(country_df_scaled)

In [ ]:
kmeans_5.labels_

In [ ]:
# assign the label
ngo_country_k['cluster_id_5'] = kmeans_5.labels_
ngo_country_k.head()

In [ ]:
# Checking the number of countries in each cluster

ngo_country_k.cluster_id_5.value_counts()

In [ ]:
## Visualisation

# Box plot on various variable against the CLUSTER_ID to visualize the spread of the data

plt.figure(figsize=(18, 5))
plt.subplot(1, 3, 1)
sns.boxplot(x='cluster_id_5', y='child_mort',
            data=ngo_country_k, palette="bright")
plt.subplot(1, 3, 2)
sns.boxplot(x='cluster_id_5', y='gdpp',
            data=ngo_country_k, palette="bright")
plt.subplot(1, 3, 3)
sns.boxplot(x='cluster_id_5', y='income',
            data=ngo_country_k, palette="bright")

plt.show()

In [ ]:
# Scatter plot on various variables to visualize the clusters based on them

plt.figure(figsize=(18, 5))
plt.subplot(1, 3, 1)
sns.scatterplot(x='gdpp', y='child_mort', hue='cluster_id_5',
                data=ngo_country_k, legend='full', palette="bright")
plt.subplot(1, 3, 2)
sns.scatterplot(x='income', y='child_mort', hue='cluster_id_5',
                data=ngo_country_k, legend='full', palette="bright")
plt.subplot(1, 3, 3)
sns.scatterplot(x='gdpp', y='income', hue='cluster_id_5',
                data=ngo_country_k, legend='full', palette="bright")
plt.show()

In [ ]:
# Checking the CLuster means:
ngo_country_k.groupby(['cluster_id_5']).mean().sort_values(['child_mort','income','gdpp'],ascending = [False,True,True])

### Observation: Cluster 3 & Cluster 1 seems to have highest Child Mortality and lowest Income & GDPP. 

## Final Interpretation: Choosing the model

### Observation: From the above 3 Iterations of K-Means, could see that using 3 Clusters provided a better output in terms of a balanced cluster size. So will consider the 'K-Means with 3 Clusters' as FINAL MODEL.

## Interpreting the Clusters from K-Means model

In [ ]:
#New dataframe for group by & analysis

ngo_country_k_analysis =  ngo_country_k.groupby(['cluster_id_3']).mean().sort_values(['child_mort','income','gdpp'],ascending = [False,True,True])
ngo_country_k_analysis

In [ ]:
# Creating a new field for count of observations in each cluster

ngo_country_k_analysis['Observations']=ngo_country_k[['cluster_id_3','child_mort']].groupby(['cluster_id_3']).count()
ngo_country_k_analysis

In [ ]:
# Creating a new field for proportion of observations in each cluster

ngo_country_k_analysis['Proportion']=round(ngo_country_k_analysis['Observations']/ngo_country_k_analysis['Observations'].sum(),2)


#Summary View
ngo_country_k_analysis[['child_mort','income','gdpp','Observations','Proportion']]

In [ ]:
plt.figure(figsize=(8, 5))
ngo_country_k_plot1=ngo_country_k[['cluster_id_3','gdpp','income']].copy()
ngo_country_k_plot1=ngo_country_k_plot1.groupby('cluster_id_3').mean()
ngo_country_k_plot1.plot.bar()

ngo_country_k_plot2=ngo_country_k[['cluster_id_3','child_mort']].copy()
ngo_country_k_plot2=ngo_country_k_plot2.groupby('cluster_id_3').mean()
ngo_country_k_plot2.plot.bar()

### Observation: Cluster 2 has the Highest average Child Mortality rate of ~92 when compared to other  clusters, and Lowest average GDPP & Income of ~ 1909 & 3897 respectively. All these figures clearly makes this cluster the best candidate for the financial aid from NGO. We could also see that Cluster 2 comprises of ~29% of overall data, and has ~48 observations in comparision to 167 total observations

### Renaming the Clusters

In [ ]:
ngo_country_k_analysis = ngo_country_k_analysis.rename({
    2: 'Under_Developed Countries',
    1: 'Developed Countries',
    0: 'Developing Countries'}).reset_index()

In [ ]:
ngo_country_k_analysis[['cluster_id_3','child_mort','income','gdpp','Observations','Proportion']]

## Analysing the 'Under Developed Countries' Cluster

In [ ]:
# Countries in Cluster_id = 3, save to a new dataframe, this will form our final cluster data

final_cluster = ngo_country_k[ngo_country_k['cluster_id_3'] == 2].copy()
final_cluster = final_cluster[['country', 'child_mort', 'income', 'gdpp']]

In [ ]:
final_cluster.sort_values([ 'gdpp', 'income','child_mort'], ascending=[True, True, False])

## Decision Making on the final approach

In [ ]:
final_cluster.describe()

#### Observation: Looking at the descriptive statistics of our Under Developed Countries cluster, we could notice that some of the countries have really high income and gdpp values (looking at the max values of income & gdpp). From a business problem prespective, we would want our numbers of child mortality, income and gdpp somewhere around the Median, as there seems to be a greater variablity in the income and gdpp values). So the approach we are going to take is to filter out all countries from our original list with income & gdpp less than the Median of 1860 & 932 respectively and Child Mortality >= the Median of 90.

#### We will perform the filtering in the order GDPP --> INCOME --> CHILD MORTALITY --- This is due to reason that, we need to identify the countries with lowest GDPP & INCOME first and then with maximum CHILD MORTALITY. This is based on the understanding that, countries with highest child mortality and having higher gdpp & income will not have any impact on the child mortality rate even after the financial aid.

## Fetching the final data based on Cluster Median
#### Data less than median of gdpp of the cluster

#### We could see that the Median values of gdpp for 'Under Developed Countries' is ~932. We will take this as a benchmark and fetch all countries from our 'ORIGINAL dataframe with gdpp <= 932.

In [ ]:
Under_Developed_Countries = country_df[country_df['gdpp'] <= 1909].copy()
# Under_Developed_Countries = Under_Developed_Countries[Under_Developed_Countries['gdpp'] <= 932]
Under_Developed_Countries=Under_Developed_Countries.sort_values([ 'gdpp', 'income','child_mort'], ascending=[True, True, False])

In [ ]:
Under_Developed_Countries[['country', 'child_mort', 'gdpp', 'income']]

In [ ]:
Under_Developed_Countries.shape

### Data less than median of income of the cluster

#### We could see that the Median values of income for 'Under Developed Countries' is ~932. We will take this as a benchmark and fetch all countries from our 'ORIGINAL dataframe with income <= 1860.

In [ ]:
Under_Developed_Countries = Under_Developed_Countries[Under_Developed_Countries['income'] <= 1860]
Under_Developed_Countries=Under_Developed_Countries.sort_values([ 'gdpp', 'income','child_mort'], ascending=[True, True, False])

In [ ]:
Under_Developed_Countries[['country', 'child_mort', 'gdpp', 'income']]

In [ ]:
Under_Developed_Countries.shape

### Data greater than median of child mortality rate of the final cluster

#### We could see that the Median values of Child Mortality for 'Under Developed Countries' is ~90. We will take this as a benchmark and fetch all countries from our 'ORIGINAL dataframe with Child Mortality >= 90.

In [ ]:
# creating a new dataframe to store the countries satisfying the cluster median from original dataframe

Under_Developed_Countries = Under_Developed_Countries[Under_Developed_Countries['child_mort'] >= 90]
Under_Developed_Countries=Under_Developed_Countries.sort_values([ 'gdpp', 'income','child_mort'], ascending=[True, True, False])

In [ ]:
Under_Developed_Countries[['country', 'child_mort', 'gdpp', 'income']]

In [ ]:
Under_Developed_Countries.shape

## Top 5 Underdeveloped Countries list

In [ ]:
# Save the top 10 underdeveloped list to a new df

UDC_top_5=Under_Developed_Countries[['country', 'gdpp', 'income', 'child_mort']].head(5).copy()

In [ ]:
#Final country list

UDC_top_5

## Conclusion- We performed CLUSTERING on the socio-economic data provided for various countries to identify countries to recommend for Financial Aid from the NGO. Based on our Clustering Analysis, we have above ,the top 5 countries under our 'Under Developed Countries' cluster which are in dire need of the Financial Aid.

## Univariate Analysis of Cluster 'Under_Developed_Countries' (Top 5)


In [ ]:
plt.figure(figsize=(21, 18))
features = ['gdpp','income','child_mort']
for i in enumerate(features):
    ax = plt.subplot(3, 3, i[0]+1)
    sns.distplot(UDC_top_5[i[1]])

## Bivariate Analysis of Cluster 'Under_Developed_Countries' (Top 5)

In [ ]:
# Scatter plot on various variables to visualize the clusters based on them

plt.figure(figsize=(18, 10))
plt.subplot(1, 3, 1)
sns.scatterplot(x='gdpp', y='child_mort', hue='country',
                data=UDC_top_5, legend='full', palette="bright", s=300, c='lightblue')
plt.subplot(1, 3, 2)
sns.scatterplot(x='gdpp', y='income', hue='country',
                data=UDC_top_5, legend='full', palette="bright", s=300, c='lightblue')
plt.subplot(1, 3, 3)
sns.scatterplot(x='income', y='child_mort', hue='country',
                data=UDC_top_5, legend='full', palette="bright", s=300, c='lightblue')
plt.show()

## Hierarchical Clustering

In [ ]:
country_df_scaled.head()

In [ ]:
## Plotting a Dendrogram:: Single Linkage

plt.figure(figsize=(18, 8))
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('index')
plt.ylabel('distance')
mergings_s = linkage(country_df_scaled, method="single", metric='euclidean')
dendrogram(mergings_s)
plt.show()

In [ ]:
# complete linkage:

plt.figure(figsize=(18, 8))
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('index')
plt.ylabel('distance')
mergings_c = linkage(country_df_scaled, method="complete", metric='euclidean')
dendrogram(mergings_c)
plt.show()

### OBSERVATION: NOTE: From the above Dendrograms, it is evident that 'Complete Linkage' give a better cluster formation. So will use Complete linkage output for our further analysis.

In [ ]:
# 3 clusters
cluster_labels_3 = cut_tree(mergings_c, n_clusters=3).reshape(-1, )
cluster_labels_3

In [ ]:
# Create a new datafame with cluster labels to it.
ngo_country_h = country_df.copy()
ngo_country_h['cluster_labels_3'] = cluster_labels_3

ngo_country_h.head()

In [ ]:
ngo_country_h.shape

In [ ]:
# Box plot on various variable against the CLUSTER_ID to visualize the spread of the data

plt.figure(figsize=(18, 5))
plt.subplot(1, 3, 1)
sns.boxplot(x='cluster_labels_3', y='child_mort', data=ngo_country_h, palette="bright")
plt.subplot(1, 3, 2)
sns.boxplot(x='cluster_labels_3', y='gdpp', data=ngo_country_h, palette="bright")
plt.subplot(1, 3, 3)
sns.boxplot(x='cluster_labels_3', y='income', data=ngo_country_h, palette="bright")

plt.show()

In [ ]:
# Scatter plot on various variables to visualize the clusters based on them

plt.figure(figsize=(18, 5))
plt.subplot(1, 3, 1)
sns.scatterplot(x='gdpp', y='child_mort', hue='cluster_labels_3',
                data=ngo_country_h, legend='full', palette="bright")
plt.subplot(1, 3, 2)
sns.scatterplot(x='income', y='child_mort', hue='cluster_labels_3',
                data=ngo_country_h, legend='full', palette="bright")
plt.subplot(1, 3, 3)
sns.scatterplot(x='gdpp', y='income', hue='cluster_labels_3',
                data=ngo_country_h, legend='full', palette="bright")
plt.show()

In [ ]:
# Checking cluster count

ngo_country_h.cluster_labels_3.value_counts()

### Observation: Cluster 1 & 2 seems to be Developed or Developing countries, so our segmentation is good in terms of all our under developed countries are segmented under cluster 0.

## Interpreting the Clusters from Hierarchical model

In [ ]:
#New dataframe for group by & analysis
ngo_country_h_analysis =  ngo_country_h.groupby(['cluster_labels_3']).mean()
ngo_country_h_analysis

In [ ]:
# Creating a new field for count of observations in each cluster
ngo_country_h_analysis['Observations']=ngo_country_h[['cluster_labels_3','child_mort']].groupby(['cluster_labels_3']).count()
ngo_country_h_analysis

In [ ]:
plt.figure(figsize=(8, 5))
ngo_country_h_plot1=ngo_country_h[['cluster_labels_3','gdpp','income']].copy()
ngo_country_h_plot1=ngo_country_h_plot1.groupby('cluster_labels_3').mean()
ngo_country_h_plot1.plot.bar()
plt.show()

ngo_country_h_plot2=ngo_country_h[['cluster_labels_3','child_mort']].copy()
ngo_country_h_plot2=ngo_country_h_plot2.groupby('cluster_labels_3').mean()
ngo_country_h_plot2.plot.bar()
plt.show()

### Observation:  Cluster 0 has the Highest average Child Mortality rate of ~42 when compared to other clusters, and Lowest average GDPP & Income of ~ 7551 & 12641 respectively. All these figures clearly makes this cluster the best candidate for the financial aid from NGO. We could also see that Cluster 0 comprises of ~89% of overall data, and has ~148 observations in comparision to 167 total observations This seems to be a problem. This means that Hierarchical clustering is not giving us a good result as 89% of the data points are segmented into that cluster. 

### Better way of interpretation is going with K MEANS Clustering as we are able to get the final set of countries names which are in dire need for financial aid.